In [1]:
import sys
sys.path.append('../')
from plotting import CandlePlot
import pandas as pd

In [2]:
from technicals.patterns import apply_patterns

In [3]:
df = pd.read_pickle('../data/GBP_JPY_H1.pkl')

In [4]:
df_an = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c',]].copy()

df_an.tail()

,time,mid_o,mid_h,mid_l,mid_c
49742,2023-12-29 17:00:00+00:00,179.547,179.815,179.505,179.756
49743,2023-12-29 18:00:00+00:00,179.754,179.880,179.640,179.648
49744,2023-12-29 19:00:00+00:00,179.650,179.776,179.530,179.756
49745,2023-12-29 20:00:00+00:00,179.760,179.872,179.695,179.728
49746,2023-12-29 21:00:00+00:00,179.728,179.806,179.457,179.550


In [ ]:
direction = df_an['mid_c'] - df_an['mid_o']
body_size = abs(direction)
direction = [1 if x >= 0 else -1 for x in direction]
full_range = df_an['mid_h'] - df_an['mid_l']
body_perc = (body_size / full_range) * 100
body_lower = df_an[['mid_c', 'mid_o']].min(axis=1)
body_upper = df_an[['mid_c', 'mid_o']].max(axis=1)
body_bottom_perc = ((body_lower - df_an['mid_l']) / full_range * 100)
body_top_perc = 100 - (((df_an['mid_h'] - body_upper) / full_range) * 100)

In [ ]:
df_an['body_lower'] = body_lower
df_an['body_upper'] = body_upper
df_an['body_bottom_perc'] = body_bottom_perc
df_an['body_top_perc'] = body_top_perc
df_an['body_perc'] = body_perc
df_an['direction'] = direction

In [5]:
df_an = apply_patterns(df_an)

In [6]:
dfp = df_an.iloc[-10:]
cp = CandlePlot(dfp)
cp.show_plot(line_traces=['body_upper','body_lower'], height=300)

c:\ProgramData\Anaconda3\envs\py_fin\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [7]:
df_an.iloc[-10]

time                2023-12-29 12:00:00+00:00
mid_o                                 180.417
mid_h                                 180.417
mid_l                                 179.822
mid_c                                 179.955
body_lower                            179.955
body_upper                            180.417
body_bottom_perc                    22.352941
body_top_perc                           100.0
body_perc                           77.647059
direction                                  -1
Name: 49737, dtype: object

In [8]:
HANGING_MAN_BODY = 15.0
HANGING_MAN_HEIGHT = 75.0

def apply_hanging_man(row):
    if row['body_bottom_perc'] > HANGING_MAN_HEIGHT:
        if row['body_perc'] < HANGING_MAN_BODY:
            return True
    return False

In [9]:
df_an['HANGING_MAN'] = df_an.apply(apply_hanging_man, axis=1)

In [12]:
import plotly.graph_objects as go

In [15]:
MARKER = '#0066FF'
dfp = df_an.iloc[-250:]
cp = CandlePlot(dfp, candles=True)

df_temp = cp.df_plot[cp.df_plot['HANGING_MAN'] == True]

cp.fig.add_trace(go.Candlestick(
                x=df_temp.time,
                open=df_temp.mid_o,
                high=df_temp.mid_h,
                low=df_temp.mid_l,
                close=df_temp.mid_c,
                line=dict(width=1),
                opacity=1,
                increasing_fillcolor=MARKER,
                decreasing_fillcolor=MARKER,
                increasing_line_color=MARKER,
                decreasing_line_color=MARKER
            ))

cp.show_plot()